In [ ]:
import torch
import os
import utils
from datetime import datetime
from data_roboflow import YoloRoboflowDataset
from models import YOLOv1
from torch.utils.data import DataLoader

# Configuration parameters
model_dir = None  # Directory containing model weights
weights = 'final'  # Name of weights file to use (without extension)
batch_size = 8  # Batch size for evaluation
iou_threshold = 0.5  # IoU threshold for mAP calculation
confidence_threshold = 0.1  # Confidence threshold for detections

# Set model directory if not provided
if model_dir is None:
    # Find most recent model directory
    base_dir = 'models/yolo_v1'
    if os.path.exists(base_dir):
        dates = os.listdir(base_dir)
        if dates:
            latest_date = max(dates)
            times = os.listdir(os.path.join(base_dir, latest_date))
            if times:
                latest_time = max(times)
                model_dir = os.path.join(base_dir, latest_date, latest_time)
                print(f"Using most recent model directory: {model_dir}")
# Assert that model directory exists
assert model_dir is not None and os.path.exists(model_dir), "No model directory found. Please specify model_dir manually"

# Load class names
classes = utils.load_class_array()
print(f"Loaded {len(classes)} classes: {classes}")

# Create dataset and dataloader
print("Loading validation dataset...")
dataset = YoloRoboflowDataset('valid', normalize=True, augment=False)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Load model
print(f"Loading model from {model_dir}...")
model = YOLOv1()
model.eval()
weights_path = os.path.join(model_dir, 'weights', weights)
model.load_state_dict(torch.load(weights_path, map_location=utils.device))
model = model.to(utils.device)

In [ ]:
import torch
import matplotlib.pyplot as plt
import supervision as sv
import cv2
from tqdm import tqdm
import numpy as np

# Get a single image
random_idx = torch.randint(0, len(dataset), (1,)).item()
image, label, original = dataset[random_idx]

# Add batch dimension
image = image.unsqueeze(0).to(utils.device)

# Get predictions
model.eval()
with torch.no_grad():
    predictions = model(image)

# Convert original image to numpy array for supervision visualization
original_np = original.permute(1, 2, 0).numpy()
original_np = (original_np * 255).astype('uint8')

# Get bounding boxes from predictions
pred_boxes = utils.extract_bboxes_from_yolo_output(predictions[0], min_confidence=0.1)
pred_boxes = utils.non_max_suppression(pred_boxes, iou_threshold=0.5)

pred_boxes


In [ ]:
# Create results directory if needed
results_dir = os.path.join('results', 'evaluation', datetime.now().strftime('%Y%m%d_%H%M%S'))
os.makedirs(results_dir, exist_ok=True)

# Evaluate model
print(f"Evaluating model on {len(dataset)} validation images...")
results = utils.evaluate_model(model, loader, classes)

# Print results
print("\nEvaluation Results:")
print("-" * 50)

# Print mAP metrics
mAP_metrics = [key for key in results.keys() if key.startswith('mAP')]
for metric in mAP_metrics:
    print(f"{metric}: {results[metric]:.4f}")

print("\nPer-class Average Precision:")
print("-" * 50)

# Print AP metrics for each class
for class_idx, class_name in enumerate(classes):
    ap_05 = results.get(f"AP@0.5_{class_name}", 0)
    ap_075 = results.get(f"AP@0.75_{class_name}", 0)
    print(f"{class_name}: AP@0.5={ap_05:.4f}, AP@0.75={ap_075:.4f}")